<h1 align="center"> SPAM Detector: Text Classify Others Aproachs</h1>

<img src="https://raw.githubusercontent.com/deepankarkotnala/Email-Spam-Ham-Classifier-NLP/master/images/email_spam_ham.png" width="50%" />

Created: 2020-09-15

Last updated: 2020-09-15

Kaggle Kernel made by 🚀 <a href="https://www.kaggle.com/rafanthx13"> Rafael Morais de Assis</a>

This is an initial kernel, in the future it will be filled with the theoretical part and more details about NLP

## Table Of Contents (TOC) <a id="top"></a>

+ [Import Libs and DataSet](#index01) 
+ [Snippets](#index02)

## Import Libs and DataSet <a id='index01'></a>

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Configs
pd.options.display.float_format = '{:,.4f}'.format
sns.set(style="whitegrid")
plt.style.use('seaborn')
seed = 42
np.random.seed(seed)

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [81]:
file_path = '/kaggle/input/sms-spam-collection-dataset/spam.csv'
df = pd.read_csv(file_path, encoding='latin-1')
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

print("DataSet = {} rows and {} columns".format(df.shape[0], df.shape[1]))
df.head()

DataSet = 5572 rows and 2 columns


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Snippets <a id='index02'></a> <a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white; margin-left: 20px;" data-toggle="popover">Go to TOC</a>

In [82]:
import time

def time_spent(time0):
    t = time.time() - time0
    t_int = int(t) // 60
    t_min = t % 60
    if(t_int != 0):
        return '{}min {:.3f}s'.format(t_int, t_min)
    else:
        return '{:.3f}s'.format(t_min)

In [83]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score

this_labels = ['HAM','SPAM']
scoress = {}

def class_report(y_real, y_my_preds, name="", labels=this_labels):
    if(name != ''):
        print(name,"\n")
    print(confusion_matrix(y_real, y_my_preds), '\n')
    print(classification_report(y_real, y_my_preds, target_names=labels))
    scoress[name] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred, average='macro')]
    
# Create DataFrame from Scores maded by 'class_report'
def create_df_fom_scores(scores=scoress):
    return pd.DataFrame(data= scoress.values(),
            columns=['acc','f1'],
            index=scoress.keys())

## Kaggle Course Strategie with Word2Vec

### No Text PreProcessing Cleaning

In [84]:
import spacy

# Need to load the large model to get the vectors
nlp_core = spacy.load('en_core_web_lg')

In [85]:
text = "These vectors can be used as features for machine learning models."

text_nlp = nlp_core(text)
print(type(text_nlp))

<class 'spacy.tokens.doc.Doc'>


In [86]:
# Word2Vec
with nlp_core.disable_pipes():
    vectors = np.array([ token.vector for token in  nlp_core(text)])

In [87]:
vectors.shape

(12, 300)

In [88]:
with nlp_core.disable_pipes():
    X = np.array([nlp_core(text).vector for text in df.message.values])

In [89]:
X.shape

(5572, 300)

In [90]:
from sklearn.model_selection import train_test_split


y = df['label'].replace({'ham': 0, 'spam': 1}).values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [91]:
from sklearn.svm import LinearSVC

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=42)#, dual=False, max_iter=10000)
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

class_report(y_test, y_pred, 'Kaggle Course: No text Cleaning Word2Vec spaCy with LinearSVC')

Kaggle Course: No text Cleaning Word2Vec spaCy with LinearSVC 

[[947  18]
 [ 14 136]] 

              precision    recall  f1-score   support

         HAM       0.99      0.98      0.98       965
        SPAM       0.88      0.91      0.89       150

    accuracy                           0.97      1115
   macro avg       0.93      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115



### With Text PreProcessing Cleaning

In [92]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
"couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
"hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would",
"I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have",
"i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
"i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
"it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
"mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
"o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
"sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
"she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
"shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
"this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
"there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
"they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
"they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
"weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
"what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
"where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
"who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
"why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
"y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
"y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will",
"you'll've": "you will have", "you're": "you are", "you've": "you have" }

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora',
'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are',
'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I',
'theBest': 'the best', 'howdoes': 'how does', 'Etherium': 'Ethereum', 'narcissit': 'narcissist',
'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend',
'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization',
'demonitization': 'demonetization', 'demonetisation': 'demonetization', 'pokémon': 'pokemon'}

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [93]:
from nltk.corpus import stopwords
import re

# Stop Words in Python. Is better search in a 'set' structure
stops = set(stopwords.words("english"))  

def clean_text( text ):
    # 1. Remove non-letters        
    text = re.sub("[^a-zA-Z]", " ", text) 
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    # 2. Convert to lower case, split into individual words
    text = text.lower().split()                                             
    # 3. Remove stop words
    meaningful_words = [w for w in text if not w in stops]   
    # 4. Join the words back into one string separated by space.
    return( " ".join( meaningful_words )) 

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lematizer(text):
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = " ".join(text)
    return text

In [94]:
clean_column = 'clean_message'

df[clean_column] = df['message'].apply(lambda x: clean_contractions(x, contraction_mapping))

df[clean_column] = df[clean_column].apply(lambda x: correct_spelling(x, mispell_dict))

df[clean_column] = df[clean_column].apply(clean_text)

df[clean_column] = df[clean_column].apply(lematizer)

In [95]:
X = df[clean_column].values
y = df['label'].replace({'ham': 0, 'spam': 1})

In [96]:
with nlp_core.disable_pipes():
    X_vec = np.array([nlp_core(text).vector for text in X])

In [97]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.20, random_state=42)

In [98]:
from sklearn.svm import LinearSVC

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=42)
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

class_report(y_test, y_pred, 'Kaggle Course: With text Cleaning Word2Vec spaCy with LinearSVC')

Kaggle Course: With text Cleaning Word2Vec spaCy with LinearSVC 

[[946  19]
 [ 22 128]] 

              precision    recall  f1-score   support

         HAM       0.98      0.98      0.98       965
        SPAM       0.87      0.85      0.86       150

    accuracy                           0.96      1115
   macro avg       0.92      0.92      0.92      1115
weighted avg       0.96      0.96      0.96      1115



## HSE Course NLP

In [99]:
df.head()

,label,message,clean_message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

XX = df['clean_message'].values
y = df['label'].replace({'ham': 0, 'spam': 1})

x_train, x_test, y_train, y_test = train_test_split(XX, y, test_size=0.20, random_state=42)

tfidf_transformer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
tfidf_transformer.fit(x_train)

x_train_tfidf = tfidf_transformer.transform(x_train)
x_test_tfidf  = tfidf_transformer.transform(x_test)

In [101]:
from sklearn.svm import LinearSVC

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=42)#, dual=False, max_iter=10000)
svc.fit(x_train_tfidf, y_train)

y_pred = svc.predict(x_test_tfidf)

class_report(y_test, y_pred, 'HSE NLP: TD-IDF LinearSVC no HyperTuned')

HSE NLP: TD-IDF LinearSVC no HyperTuned 

[[960   5]
 [ 13 137]] 

              precision    recall  f1-score   support

         HAM       0.99      0.99      0.99       965
        SPAM       0.96      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Hyper Tuning LinearSVC

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

def optimize(mx_train, my_train, my_hyper_params, hyper_to_search, hyper_search_name, cv=4, scoring='accuracy'):
    """search best param to unic one hyper param
    @mx_train, @my_train = x_train, y_train of dataset
    @my_hyper_params: dict with actuals best_params: start like: {}
      => will be accumulated and modified with each optimization iteration
      => example stater: best_hyper_params = {'random_state': 42, 'n_jobs': -1}
    @hyper_to_search: dict with key @hyper_search_name and list of values to gridSearch:
    @hyper_search_name: name of hyperparam
    """
    if(hyper_search_name in my_hyper_params.keys()):
        del my_hyper_params[hyper_search_name]
    if(hyper_search_name not in hyper_to_search.keys()):
        raise Exception('"hyper_to_search" dont have {} in dict'.format(hyper_search_name))
        
    t0 = time.time()
        
    rf = LinearSVC(**my_hyper_params)
    
    grid_search = GridSearchCV(estimator = rf, param_grid = hyper_to_search, 
      scoring = scoring, n_jobs = -1, cv = cv)
    grid_search.fit(mx_train, my_train)
    
    print('took', time_spent(t0))
    
    data_frame_results = pd.DataFrame(
        data={'mean_fit_time': grid_search.cv_results_['mean_fit_time'],
        'mean_test_score_'+scoring: grid_search.cv_results_['mean_test_score'],
        'ranking': grid_search.cv_results_['rank_test_score']
         },
        index=grid_search.cv_results_['params']).sort_values(by='ranking')
    
    print('The Best HyperParam to "{}" is {} with {} in {}'.format(
        hyper_search_name, grid_search.best_params_[hyper_search_name], grid_search.best_score_, scoring))
    
    my_hyper_params[hyper_search_name] = grid_search.best_params_[hyper_search_name]
    
    """
    @@my_hyper_params: my_hyper_params appends best param find to @hyper_search_name
    @@data_frame_results: dataframe with statistics of gridsearch: time, score and ranking
    @@grid_search: grid serach object if it's necessary
    """
    return my_hyper_params, data_frame_results, grid_search

In [103]:
best_hyper_params = {'random_state': 42} # Stater Hyper Params

In [104]:
search_hyper = {'penalty': ['l1', 'l2']}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'penalty')

took 2.856s
The Best HyperParam to "penalty" is l2 with 0.9854167505293412 in accuracy


In [105]:
search_hyper = {'loss': ['hinge', 'squared_hinge']}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'loss')

took 0.091s
The Best HyperParam to "loss" is squared_hinge with 0.9854167505293412 in accuracy


In [106]:
search_hyper = {'dual': [True, False]}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'dual')


took 0.082s
The Best HyperParam to "dual" is True with 0.9854167505293412 in accuracy


In [107]:
search_hyper = {'C': [0.8, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 20.0, 80.0]}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'C')


took 0.445s
The Best HyperParam to "C" is 1.4 with 0.9858653822930337 in accuracy


In [108]:
best_hyper_params

{'random_state': 42,
 'penalty': 'l2',
 'loss': 'squared_hinge',
 'dual': True,
 'C': 1.4}

In [109]:
search_hyper = {'max_iter': [1000, 16000]}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'max_iter')


took 0.080s
The Best HyperParam to "max_iter" is 1000 with 0.9858653822930337 in accuracy


In [110]:
search_hyper = {'tol': [0.00006, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0 ]}

best_hyper_params, results, last_grid_search = optimize(
    x_train_tfidf, y_train, best_hyper_params, search_hyper, 'tol')


took 0.180s
The Best HyperParam to "tol" is 6e-05 with 0.9858653822930337 in accuracy


In [112]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=42)
svc.fit(x_train_tfidf, y_train)

y_pred = svc.predict(x_test_tfidf)

class_report(y_test, y_pred, 'HSE NLP: TD-IDF LinearSVC NO HyperTuned')

HSE NLP: TD-IDF LinearSVC NO HyperTuned 

[[960   5]
 [ 13 137]] 

              precision    recall  f1-score   support

         HAM       0.99      0.99      0.99       965
        SPAM       0.96      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [113]:
from sklearn.svm import LinearSVC

svc = LinearSVC(**best_hyper_params)
svc.fit(x_train_tfidf, y_train)

y_pred = svc.predict(x_test_tfidf)

class_report(y_test, y_pred, 'HSE NLP: TD-IDF LinearSVC HyperTuned')

HSE NLP: TD-IDF LinearSVC HyperTuned 

[[959   6]
 [ 11 139]] 

              precision    recall  f1-score   support

         HAM       0.99      0.99      0.99       965
        SPAM       0.96      0.93      0.94       150

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [114]:
create_df_fom_scores()

,acc,f1
Kaggle Course: No text Cleaning Word2Vec spaCy with LinearSVC,0.9713,0.9391
Kaggle Course: With text Cleaning Word2Vec spaCy with LinearSVC,0.9632,0.9204
HSE NLP: TD-IDF LinearSVC no HyperTuned,0.9839,0.9645
HSE NLP: TD-IDF LinearSVC NO HyperTuned,0.9839,0.9645
HSE NLP: TD-IDF LinearSVC HyperTuned,0.9848,0.9668


## Spacy Text Classifyer Pipiline

In [124]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
                "textcat",
                config={
                    "exclusive_classes": True,
                    "architecture": "bow"})
# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

# Add NEGATIVE and POSITIVE labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

In [125]:
from spacy.util import minibatch
import random

def train(model, train_data, optimizer, batch_size=8):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size=batch_size)
    for batch in batches:
        texts, labels = zip(*batch)
        model.update(texts, labels, sgd=optimizer, losses=losses)
    return losses

In [ ]:
# Fix seed for reproducibility
spacy.util.fix_random_seed(1)
random.seed(1)

# This may take a while to run!
optimizer = nlp.begin_training()
train_data = list(zip(train_texts, train_labels))
losses = train(nlp, train_data, optimizer)
print(losses['textcat'])

In [ ]:
text = "This tea cup was full of holes. Do not recommend."
doc = nlp(text)
print(doc.cats)

In [ ]:
def predict(nlp, texts):
    # Use the tokenizer to tokenize each input text example
    docs = [nlp.tokenizer(text) for text in texts]

    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe('textcat')
    scores, _ = textcat.predict(docs)

    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)

    return predicted_class

In [ ]:
texts = val_texts[34:38]
predictions = predict(nlp, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

In [ ]:
def evaluate(model, texts, labels):
    """ Returns the accuracy of a TextCategorizer model. 
    
        Arguments
        ---------
        model: ScaPy model with a TextCategorizer
        texts: Text samples, from load_data function
        labels: True labels, from load_data function
    
    """
    # Get predictions from textcat model
    predicted_class = predict(model, texts)

    # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
    true_class = [int(each['cats']['POSITIVE']) for each in labels]

    # A boolean or int array indicating correct predictions
    correct_predictions = predicted_class == true_class

    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()

    return accuracy

In [ ]:
accuracy = evaluate(nlp, val_texts, val_labels)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
# This may take a while to run! # DEMORA MUITO
n_iters = 3
for i in range(n_iters):
    losses = train(nlp, train_data, optimizer)
    accuracy = evaluate(nlp, val_texts, val_labels)
    print(f"Loss: {losses['textcat']:.3f} \t Accuracy: {accuracy:.3f}")